![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/track-and-monitor-experiments/using-mlflow/train-projects-remote/train-projects-remote.png)

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Train with MLflow Projects on AML Compute

Train MLflow Projects on Azure Machine Learning Compute.

Train MLflow Projects on your machine with AzureML compute and tracking. In this notebook you will:

1. Set up MLflow tracking URI to track experiments and metrics in AzureML
2. Create experiment
3. Set up an MLflow project to run on AzureML compute
4. Submit an MLflow project run and view it in an AzureML workspace 


## Prerequisites 

If you are using a Notebook VM, you are all set. Otherwise, go through the [Configuration](../../../../configuration.ipnyb) notebook to set up your Azure Machine Learning workspace and ensure other common prerequisites are met.

Make sure you have the following before staring the notebook: 
- Connected to an AML Workspace 
- Have an existing [Azure ML Compute cluster](https://docs.microsoft.com/azure/machine-learning/how-to-create-attach-compute-sdk#amlcompute) in that Workspace 
- Have an MLproject file with a modified environment specification 

Add the azureml-mlflow package as a pip dependency to your environment configuration file (conda.yaml). The project can run without this addition, but key artifacts and metrics will not be logged to your Workspace. An example conda.yaml is included in this tutorial folder with the necessary packages. 


## Set-up 

Check the Azure ML and MLflow SDK version installed on your computer and connect to your workspace

In [1]:
import sys, os
import mlflow
import mlflow.azureml

import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()

print("SDK version:", azureml.core.VERSION)
print("MLflow version:", mlflow.version.VERSION)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (cryptography 3.2 (/home/krbock/anaconda3/envs/azureml/lib/python3.7/site-packages), Requirement.parse('cryptography<4.0.0,>=3.3.1; extra == "crypto"'), {'PyJWT'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (cryptography 3.2 (/home/krbock/anaconda3/envs/azureml/lib/python3.7/site-packages), Requirement.parse('cryptography<4.0.0,>=3.3.1; extra == "crypto"'), {'PyJWT'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (cryptography 3.2 (/home/krbock/anaconda3/envs/azureml/lib/python3.7/site-packages), Requirement.parse('cryptography<4.0.0,>=3.3.1; extra == "crypto"'), {'PyJWT'}).
Failure while loading azureml_run_type

SDK version: 1.39.0
MLflow version: 1.22.0
msmls
mlservices
australiaeast
3c8972d9-f541-46b2-b70b-d81baba3595d


## Initialize Tracking Store and Experiment

### Set Tracking URI 

Set the MLflow tracking URI to point to your Azure ML Workspace. The subsequent logging calls from MLflow APIs will go to Azure ML services and will be tracked under your Workspace.

In [2]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

### Create Experiment

Create an Mlflow Experiment to organize your runs. It can be set either by passing the name as a **parameter** in the mlflow.projects.run call or by the following,

In [3]:
experiment_name = "train-project-amlcompute"
mlflow.set_experiment(experiment_name)

2022/06/06 21:34:35 INFO mlflow.tracking.fluent: Experiment with name 'train-project-amlcompute' does not exist. Creating a new experiment.


<Experiment: artifact_location='', experiment_id='c8c89c83-6b41-4fbc-a614-5ea5d4636e13', lifecycle_stage='active', name='train-project-amlcompute', tags={}>

## Create the Backend Configuration Object

The backend configuration object will store necesary information for the integration such as the compute target and whether to use your local managed environment or a system managed environment. 

The integration will accept "COMPUTE" and "USE_CONDA" as parameters where "COMPUTE" is set to the name of your remote compute cluster and "USE_CONDA" which creates a new environment for the project from the environment configuration file. If "COMPUTE" is present in the object, the project will be automatically submitted to the remote compute and ignore "USE_CONDA". MLflow accepts a dictionary object or a JSON file.

In [4]:
# dictionary
backend_config = {"COMPUTE": "cpu-cluster", "USE_CONDA": False}


## Modify your Environment specification

Add the azureml-mlflow package as a pip dependency to your environment configuration file (conda.yaml). The project can run without this addition, but key artifacts and metrics will not be logged to your Workspace. An example conda.yaml is included in the notebook folder. Adding it to to the file will look like this,

```
name: mlflow-example
channels:
  - defaults
  - anaconda
  - conda-forge
dependencies:
  - python=3.6
  - scikit-learn=0.19.1
  - pip
  - pip:
    - mlflow
    - azureml-mlflow
```

## Submit Run 

Submit the mlflow project run using aml compute and ensure the **backened** parameter is set to azureml.



In [5]:
remote_mlflow_run = mlflow.projects.run(uri=".", 
                                    parameters={"alpha":0.3},
                                    backend = "azureml",
                                    backend_config = backend_config,
                                    synchronous=True)

Class AzureMLProjectBackend: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2022/06/06 21:34:38 INFO mlflow.projects.utils: === Created directory /tmp/tmp67277z90 for downloading remote URIs passed to arguments of type 'path' ===
No Python version provided, defaulting to "3.6.2"
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.
Class AzureMLSubmittedRun: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


RunId: train-project-amlcompute_1654515283_21e70327
Web View: https://ml.azure.com/runs/train-project-amlcompute_1654515283_21e70327?wsid=/subscriptions/3c8972d9-f541-46b2-b70b-d81baba3595d/resourcegroups/mlservices/workspaces/msmls&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Streaming azureml-logs/20_image_build_log.txt

2022/06/06 11:34:49 Downloading source code...
2022/06/06 11:34:50 Finished downloading source code
2022/06/06 11:34:50 Creating Docker network: acb_default_network, driver: 'bridge'
2022/06/06 11:34:50 Successfully set up Docker network: acb_default_network
2022/06/06 11:34:50 Setting up Docker configuration...
2022/06/06 11:34:51 Successfully set up Docker configuration
2022/06/06 11:34:51 Logging in to registry: msmls5213556472.azurecr.io
2022/06/06 11:34:55 Successfully logged into msmls5213556472.azurecr.io
2022/06/06 11:34:55 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2022/06/06 11:34:55 Scanning for de

2022/06/06 21:48:41 INFO mlflow.projects: === Run (ID 'train-project-amlcompute_1654515283_21e70327') succeeded ===


### View run 


In [6]:
remote_mlflow_run

Experiment,Id,Type,Status,Details Page,Docs Page
train-project-amlcompute,train-project-amlcompute_1654515283_21e70327,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Next Steps

Try out these notebooks to learn more about MLflow-Azure Machine Learning integration:

 * [Train a model using remote compute on Azure Cloud](../train-remote/train-remote.ipynb)
 * [Train a model using Pytorch and MLflow](../../../ml-frameworks/using-mlflow/train-and-deploy-pytorch)

